In [9]:
import sqlite3
import os
from collections import defaultdict

# --- Configuration ---
# Make sure this points to your actual database file
DATABASE_FILE = 'venues-iclr-2025-v2.db'

# --- Helper Functions (Unchanged) ---
def calculate_percentage(count, total):
    """Calculates percentage, handling division by zero."""
    if total == 0:
        return 0.0
    return (count / total) * 100

def format_output(label, absolute, percentage):
    """Formats the output string."""
    return f"{label}: {absolute} ({percentage:.2f}%)"

def is_affiliated(country):
    """
    Checks if an affiliation country is considered valid.
    A country is NOT valid if it is NULL, empty/whitespace, or 'UNK' (case-insensitive).
    """
    if country is None:
        return False
    country_stripped = country.strip()
    if not country_stripped:
        return False
    if country_stripped.upper() == 'UNK':
        return False
    return True

# --- Modified Main Logic ---
def analyze_affiliations_by_venue(db_path):
    """
    Connects to the database, fetches data for accepted papers,
    groups by venue (conference, year, track), and calculates
    affiliation statistics for each venue.
    """
    if not os.path.exists(db_path):
        print(f"Error: Database file not found at {db_path}")
        return None

    conn = None
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Fetch all venue definitions first
        cursor.execute("SELECT id, conference, year, track FROM venue_infos")
        venue_rows = cursor.fetchall()
        if not venue_rows:
            print("No venues found in the venue_infos table.")
            return {} # Return empty dict if no venues

        venue_details_map = {
            # Maps venue_info_id to (conference, year, track) tuple
            v_id: (conf, year, track) for v_id, conf, year, track in venue_rows
        }
        venue_keys = list(venue_details_map.values()) # Get a list of (conf, year, track) tuples

        # Fetch paper_id, author_id, affiliation_country, and venue_info_id
        # for all authors on papers with status 'accepted'.
        # Join with papers and venue_infos to get venue id.
        query = """
        SELECT
            pa.paper_id,
            pa.author_id,
            pa.affiliation_country,
            p.venue_info_id -- Added venue_info_id
        FROM paper_authors AS pa
        JOIN papers AS p ON pa.paper_id = p.id
        WHERE LOWER(p.status) = 'accepted';
        """
        cursor.execute(query)
        rows = cursor.fetchall()

        if not rows:
            print("No accepted papers with author information found.")
            # Initialize empty stats for all known venues
            results_by_venue = {}
            for v_key in venue_keys:
                 results_by_venue[v_key] = {
                    "total_accepted_papers": 0, "total_authors_on_accepted": 0,
                    "authors_with_country": 0, "authors_without_country": 0,
                    "papers_at_least_one_unaffiliated": 0, "papers_all_unaffiliated": 0,
                    "papers_at_least_one_affiliated": 0, "papers_all_affiliated": 0,
                    "paper_ids_all_unaffiliated": []
                }
            return results_by_venue


        # --- Data Processing per Venue ---
        # Structure: venue_data[(conf, year, track)][paper_id] = list of (author_id, country)
        venue_paper_data = defaultdict(lambda: defaultdict(list))
        # Structure: venue_authors[(conf, year, track)] = set of author_ids
        venue_authors = defaultdict(set)
        # Structure: venue_papers[(conf, year, track)] = set of paper_ids
        venue_papers = defaultdict(set)
        # Structure: venue_affiliated_authors[(conf, year, track)] = set of author_ids with country
        venue_affiliated_authors = defaultdict(set)


        for paper_id, author_id, country, venue_info_id in rows:
            if venue_info_id not in venue_details_map:
                 # Should not happen if DB integrity is maintained, but good to check
                 # print(f"Warning: venue_info_id {venue_info_id} for paper {paper_id} not found in venue_infos.")
                 continue

            venue_key = venue_details_map[venue_info_id] # (conference, year, track)

            venue_paper_data[venue_key][paper_id].append((author_id, country))
            venue_authors[venue_key].add(author_id)
            venue_papers[venue_key].add(paper_id)

            if is_affiliated(country):
                venue_affiliated_authors[venue_key].add(author_id)

        # --- Calculate Statistics per Venue ---
        results_by_venue = {}

        # Initialize results for ALL venues found in venue_infos, even if they have no accepted papers
        for v_key in venue_keys:
            results_by_venue[v_key] = {
                "total_accepted_papers": 0, "total_authors_on_accepted": 0,
                "authors_with_country": 0, "authors_without_country": 0,
                "papers_at_least_one_unaffiliated": 0, "papers_all_unaffiliated": 0,
                "papers_at_least_one_affiliated": 0, "papers_all_affiliated": 0,
                "paper_ids_all_unaffiliated": []
            }

        # Now populate stats for venues that DO have data
        for venue_key, papers_dict in venue_paper_data.items():
            stats = results_by_venue[venue_key] # Get the specific venue's stats dict

            # Calculate Author Statistics for this venue
            stats["total_authors_on_accepted"] = len(venue_authors[venue_key])
            stats["authors_with_country"] = len(venue_affiliated_authors[venue_key])
            stats["authors_without_country"] = stats["total_authors_on_accepted"] - stats["authors_with_country"]

            # Calculate Paper Statistics for this venue
            stats["total_accepted_papers"] = len(venue_papers[venue_key])
            paper_ids_all_unaffiliated_list = [] # Temp list for this venue

            for paper_id, authors_list in papers_dict.items():
                num_authors_on_paper = len(authors_list)
                num_affiliated_on_paper = 0
                num_unaffiliated_on_paper = 0

                if num_authors_on_paper == 0:
                    # print(f"Warning: Paper {paper_id} in venue {venue_key} has 0 authors listed in paper_authors.")
                    continue # Skip papers with no authors listed

                for _, country in authors_list:
                    if is_affiliated(country):
                        num_affiliated_on_paper += 1
                    else:
                        num_unaffiliated_on_paper += 1

                # Check paper conditions and increment venue-specific counters
                if num_unaffiliated_on_paper > 0:
                    stats["papers_at_least_one_unaffiliated"] += 1
                if num_unaffiliated_on_paper == num_authors_on_paper:
                    stats["papers_all_unaffiliated"] += 1
                    paper_ids_all_unaffiliated_list.append(paper_id) # Add to venue's list
                if num_affiliated_on_paper > 0:
                    stats["papers_at_least_one_affiliated"] += 1
                if num_affiliated_on_paper == num_authors_on_paper:
                    stats["papers_all_affiliated"] += 1

            # Sort and store the list of paper IDs for this venue
            paper_ids_all_unaffiliated_list.sort()
            stats["paper_ids_all_unaffiliated"] = paper_ids_all_unaffiliated_list

        return results_by_venue

    except sqlite3.Error as e:
        print(f"Database error: {e}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    finally:
        if conn:
            conn.close()

# --- Main Execution (Modified for Venue Breakdown) ---
if __name__ == "__main__":
    all_venue_stats = analyze_affiliations_by_venue(DATABASE_FILE)

    if all_venue_stats is None:
        print("Analysis could not be completed due to errors.")
    elif not all_venue_stats:
         print("No venues found or no accepted papers to analyze.")
    else:
        print("--- Affiliation Analysis Results by Venue ---")

        # Sort venues for consistent output order (optional but nice)
        sorted_venue_keys = sorted(all_venue_stats.keys())

        for venue_key in sorted_venue_keys:
            stats = all_venue_stats[venue_key]
            conference, year, track = venue_key

            print(f"\n\n=== Venue: {conference} {year} - Track: {track} ===")

            total_papers = stats["total_accepted_papers"]
            total_authors = stats["total_authors_on_accepted"]

            print(f"\nTotal Accepted Papers: {total_papers}")
            print(f"Total Unique Authors on Accepted Papers: {total_authors}")

            print("\n--- Author Affiliation (Country - excluding NULL, empty, 'UNK') ---")
            if total_authors > 0:
                print(format_output(
                    "Authors with Affiliation Country (at least once)",
                    stats["authors_with_country"],
                    calculate_percentage(stats["authors_with_country"], total_authors)
                ))
                print(format_output(
                    "Authors without Affiliation Country (only invalid/missing entries)",
                    stats["authors_without_country"],
                    calculate_percentage(stats["authors_without_country"], total_authors)
                ))
            else:
                print("No authors found on accepted papers for this venue.")


            print("\n--- Paper Affiliation Status (Country - excluding NULL, empty, 'UNK') ---")
            if total_papers > 0:
                print(format_output(
                    "Papers with at least 1 Unaffiliated Author",
                    stats["papers_at_least_one_unaffiliated"],
                    calculate_percentage(stats["papers_at_least_one_unaffiliated"], total_papers)
                ))
                print(format_output(
                    "Papers with ALL Authors Unaffiliated",
                    stats["papers_all_unaffiliated"],
                    calculate_percentage(stats["papers_all_unaffiliated"], total_papers)
                ))
                print(format_output(
                    "Papers with at least 1 Affiliated Author",
                    stats["papers_at_least_one_affiliated"],
                    calculate_percentage(stats["papers_at_least_one_affiliated"], total_papers)
                ))
                print(format_output(
                    "Papers with ALL Authors Affiliated",
                    stats["papers_all_affiliated"],
                    calculate_percentage(stats["papers_all_affiliated"], total_papers)
                ))

                # Print the list of papers where all authors are unaffiliated for this venue
                print("\n--- Papers with ALL Authors Unaffiliated ---")
                all_unaffiliated_ids = stats["paper_ids_all_unaffiliated"]
                if all_unaffiliated_ids:
                    print(f"Found {len(all_unaffiliated_ids)} paper(s):")
                    # Print only IDs for brevity, add titles if needed by modifying query and processing
                    id_list_str = ", ".join(map(str, all_unaffiliated_ids))
                    print(f"  IDs: {id_list_str}")
                    # for paper_id in all_unaffiliated_ids:
                    #     print(f"  - ID: {paper_id}")
                else:
                    print("No papers found where all authors are unaffiliated for this venue.")
            else: # total_papers == 0
                 print("No accepted papers found for this venue.")

        print("\n--- End of Analysis ---")

--- Affiliation Analysis Results by Venue ---


=== Venue: ICLR 2025 - Track: Conference ===

Total Accepted Papers: 3705
Total Unique Authors on Accepted Papers: 15066

--- Author Affiliation (Country - excluding NULL, empty, 'UNK') ---
Authors with Affiliation Country (at least once): 14908 (98.95%)
Authors without Affiliation Country (only invalid/missing entries): 158 (1.05%)

--- Paper Affiliation Status (Country - excluding NULL, empty, 'UNK') ---
Papers with at least 1 Unaffiliated Author: 135 (3.64%)
Papers with ALL Authors Unaffiliated: 2 (0.05%)
Papers with at least 1 Affiliated Author: 3703 (99.95%)
Papers with ALL Authors Affiliated: 3570 (96.36%)

--- Papers with ALL Authors Unaffiliated ---
Found 2 paper(s):
  IDs: XBHoaHlGQM, pqOjj90Vwp

--- End of Analysis ---
